In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import haversine
from google.cloud import bigquery
import glob
from fuzzywuzzy import fuzz

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

pd.options.display.max_columns=50

print(os.getcwd())
print(datetime.datetime.now())

/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/QC_store_list
2020-10-21 15:00:59.737483


/home/jliang/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
path_poi_data_folder="/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/"
path_brand_info="/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/brand_info/07/02/brand_info.csv"
client=bigquery.Client()

In [3]:
query_string='''
SELECT * from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num;
'''
print("query: \n%s"%query_string)
df_storelist_BQ_org=client.query(query_string).result().to_dataframe()
print("df_storelist_BQ_org.shape",df_storelist_BQ_org.shape)

query: 

SELECT * from mdb.vw__installer_master as t1
left join mdb_cdl.store t2 on t1.store_num=t2.store_num;



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


df_storelist_BQ_org.shape (5872, 63)


In [4]:
list_cols_store_keep=['installer_store_num','store_num','installer_name','company_owned_store_ind','open_date','close_date',
                      'address_1','city','state','zip_cd','country','latitude','longitude']
df_storelist_BQ=df_storelist_BQ_org[list_cols_store_keep]

# remove closed stores
print(df_storelist_BQ.shape)
df_storelist_BQ=df_storelist_BQ[pd.isnull(df_storelist_BQ['close_date'])]
# keep coorparate only
print(df_storelist_BQ.shape)
df_storelist_BQ=df_storelist_BQ[df_storelist_BQ['company_owned_store_ind']=="Y"]
print(df_storelist_BQ.shape)

df_storelist_BQ.head(3)

(5872, 13)
(4543, 13)
(571, 13)


,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1,city,state,zip_cd,country,latitude,longitude
2,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 S Sepulveda Blvd,Los Angeles,CA,90045-4805,US,33.953199,-118.395727
129,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 Glenwood Avenue,Raleigh,NC,27612-4532,US,35.837050,-78.675850
770,0000902237,2237,GDYR AUTO SERVICE CTR #2237,Y,2015-10-01,NaT,5811 Atlanta Highway,Montgomery,AL,36117-2109,US,32.382199,-86.198940


In [5]:
list_BQ_add_cols=['address_1','city','state','zip_cd','country','latitude','longitude']
for col in list_BQ_add_cols:
    df_storelist_BQ=df_storelist_BQ.rename(columns={col:col+"_BQ"})
    
df_storelist_BQ['zip_5_BQ']=df_storelist_BQ['zip_cd_BQ'].apply(lambda x: x.split("-")[0].zfill(5))
print(df_storelist_BQ['zip_5_BQ'].apply(len).unique())

for col in list_BQ_add_cols[:5]:
    df_storelist_BQ[col+"_BQ"]=df_storelist_BQ[col+"_BQ"].apply(lambda x: x.lower().strip())
print(df_storelist_BQ['country_BQ'].unique())
df_storelist_BQ['str_add_BQ']=df_storelist_BQ['address_1_BQ']+", "+df_storelist_BQ['city_BQ']+", "+df_storelist_BQ['state_BQ']+", "+df_storelist_BQ['zip_5_BQ']+", "+df_storelist_BQ['country_BQ']
df_storelist_BQ.head(5)


[5]
['us']


,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ
2,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 s sepulveda blvd,los angeles,ca,90045-4805,us,33.953199,-118.395727,90045,"9210 s sepulveda blvd, los angeles, ca, 90045, us"
129,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 glenwood avenue,raleigh,nc,27612-4532,us,35.837050,-78.675850,27612,"4325 glenwood avenue, raleigh, nc, 27612, us"
770,0000902237,2237,GDYR AUTO SERVICE CTR #2237,Y,2015-10-01,NaT,5811 atlanta highway,montgomery,al,36117-2109,us,32.382199,-86.198940,36117,"5811 atlanta highway, montgomery, al, 36117, us"
771,0000902382,2382,GDYR AUTO SERVICE CTR #2382,Y,2015-08-20,NaT,910 concord parkway north,concord,nc,28027-5943,us,35.424506,-80.615242,28027,"910 concord parkway north, concord, nc, 28027, us"
774,0000906740,6740,GDYR AUTO SERVICE CTR #6740,Y,2015-07-23,NaT,7320 north keystone avenue,indianapolis,in,46240-3246,us,39.888791,-86.122685,46240,"7320 north keystone avenue, indianapolis, in, ..."


In [6]:
# POI

df_POI_brand=pd.read_csv(path_brand_info,dtype=str)
list_GY_brand=[x for x in df_POI_brand['brand_name'].tolist() if "good" in x.lower() and "year" in x.lower()]
list_GY_brand

['Goodyear Auto Service Centers',
 'Goodyear Commercial Tire & Service Center',
 'Goodyear Tire & Rubber Co']

In [7]:
list_poi_files=glob.glob(path_poi_data_folder+"*.csv*")
list_poi_files.sort()
list_poi_files

['/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part1.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part10.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part11.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part2.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part3.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part4.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part5.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part6.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/POI/202010_POI/07/02/core_poi-geometry-part7.csv.gz',
 '/mnt/clients/juba/hqjubaapp02/sha

In [8]:
df_POI_GY=pd.DataFrame()
for file in list_poi_files:
    df=pd.read_csv(file,compression="gzip",dtype=str)
    df=df[df['brands'].isin(list_GY_brand)]
    df_POI_GY=df_POI_GY.append(df)
print(df_POI_GY.shape)
print(df_POI_GY['safegraph_place_id'].nunique())
df_POI_GY.groupby("brands")['safegraph_place_id'].count().to_frame().reset_index()

(658, 29)
658


,brands,safegraph_place_id
0,Goodyear Auto Service Centers,464
1,Goodyear Commercial Tire & Service Center,194


In [9]:
df_POI_GY=df_POI_GY[['safegraph_place_id','location_name','brands',
                     'street_address','city','region','postal_code','iso_country_code','latitude','longitude']]
df_POI_GY['zip_5_POI']=df_POI_GY['postal_code'].apply(lambda x: x.split("-")[0].zfill(5))
print(df_POI_GY['zip_5_POI'].apply(len).unique())

for col in ['street_address','city','region','zip_5_POI','iso_country_code']:
    df_POI_GY[col]=df_POI_GY[col].apply(lambda x: x.lower().strip())
print(df_POI_GY['iso_country_code'].unique())
df_POI_GY['str_add_POI']=df_POI_GY['street_address']+", "+df_POI_GY['city']+", "+df_POI_GY['region']+", "+df_POI_GY['zip_5_POI']+", "+df_POI_GY['iso_country_code']
df_POI_GY.head(5)

[5]
['us']


,safegraph_place_id,location_name,brands,street_address,city,region,postal_code,iso_country_code,latitude,longitude,zip_5_POI,str_add_POI
6333,sg:b9e7806050a04aaf8125dca2dfc9f49c,Goodyear Auto Service Centers,Goodyear Auto Service Centers,6411 old monroe rd,indian trail,nc,28079,us,35.049561,-80.648456,28079,"6411 old monroe rd, indian trail, nc, 28079, us"
7408,sg:072add95f12745469488aa871c4dee86,Goodyear Auto Service Centers,Goodyear Auto Service Centers,7138 s padre island dr,corpus christi,tx,78412,us,27.694699,-97.342275,78412,"7138 s padre island dr, corpus christi, tx, 78..."
8983,sg:717c10aa66354be0b542557677718e59,Goodyear Auto Service Centers,Goodyear Auto Service Centers,1630 w alexis rd,toledo,oh,43612,us,41.721714,-83.5844,43612,"1630 w alexis rd, toledo, oh, 43612, us"
26123,sg:1d47ef76ed434f05890c963c0840f6d7,Goodyear Auto Service Centers,Goodyear Auto Service Centers,9927 atlantic blvd,jacksonville,fl,32225,us,30.323003,-81.541298,32225,"9927 atlantic blvd, jacksonville, fl, 32225, us"
31806,sg:9b07df2bb145457da30bf98db59f3fb7,Goodyear Auto Service Centers,Goodyear Auto Service Centers,14275 collier blvd,naples,fl,34119,us,26.246843,-81.689046,34119,"14275 collier blvd, naples, fl, 34119, us"


In [10]:
dict_POI_add=df_POI_GY.set_index("safegraph_place_id").to_dict()['str_add_POI']
len(dict_POI_add)

658

In [11]:
dict_POI_add

{'sg:b9e7806050a04aaf8125dca2dfc9f49c': '6411 old monroe rd, indian trail, nc, 28079, us',
 'sg:072add95f12745469488aa871c4dee86': '7138 s padre island dr, corpus christi, tx, 78412, us',
 'sg:717c10aa66354be0b542557677718e59': '1630 w alexis rd, toledo, oh, 43612, us',
 'sg:1d47ef76ed434f05890c963c0840f6d7': '9927 atlantic blvd, jacksonville, fl, 32225, us',
 'sg:9b07df2bb145457da30bf98db59f3fb7': '14275 collier blvd, naples, fl, 34119, us',
 'sg:e48d227538de4f568502fe5225c064ee': '6820 old dominion dr, mclean, va, 22101, us',
 'sg:2c0527b63cb54268ae4451b763dc163d': '5725 johnston st, lafayette, la, 70503, us',
 'sg:b76fda323eba4ba7a3188d1b442cede0': '91 590 farrington highway, kapolei, hi, 96707, us',
 'sg:486afe771e904a93a29393448fd9c852': '3815 nicols rd, eagan, mn, 55122, us',
 'sg:aeb386808bc442898fc43f6a9fca7bf9': '1790 highway 160 w, fort mill, sc, 29708, us',
 'sg:11ce8fba58f04291ac972925083c35f2': '3151 lincoln hwy, thorndale, pa, 19372, us',
 'sg:5132b7e0ad124bbf990dbe141f0f

In [12]:
df_storelist_BQ.head(2)

,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ
2,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 s sepulveda blvd,los angeles,ca,90045-4805,us,33.953199,-118.395727,90045,"9210 s sepulveda blvd, los angeles, ca, 90045, us"
129,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 glenwood avenue,raleigh,nc,27612-4532,us,35.837050,-78.675850,27612,"4325 glenwood avenue, raleigh, nc, 27612, us"


In [13]:
df_storelist_BQ=df_storelist_BQ.reset_index()
del df_storelist_BQ['index']
df_storelist_BQ['poi_id']=np.nan
df_storelist_BQ['poi_add']=np.nan
df_storelist_BQ['fuzz_ratio']=np.nan

for ind,row in df_storelist_BQ.iterrows():
    add_BQ=row['str_add_BQ']
    temp_max_score=0
    best_match_k=np.nan
    best_match_v=np.nan
    
    for k,v in dict_POI_add.items():
        score=fuzz.ratio(add_BQ,v)
        if score>temp_max_score:
            temp_max_score=score
            best_match_k=k
            best_match_v=v
    if temp_max_score>95:
        del dict_POI_add[best_match_k]
    df_storelist_BQ.loc[ind,'poi_id']=best_match_k
    df_storelist_BQ.loc[ind,'poi_add']=best_match_v
    df_storelist_BQ.loc[ind,'fuzz_ratio']=temp_max_score
        
df_storelist_BQ.head(5)      

,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ,poi_id,poi_add,fuzz_ratio
0,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 s sepulveda blvd,los angeles,ca,90045-4805,us,33.953199,-118.395727,90045,"9210 s sepulveda blvd, los angeles, ca, 90045, us",sg:10999a1417bf432e818360b377cbb134,"15751 los gatos blvd, los gatos, ca, 95032, us",63.0
1,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 glenwood avenue,raleigh,nc,27612-4532,us,35.837050,-78.675850,27612,"4325 glenwood avenue, raleigh, nc, 27612, us",sg:0bb487af91b544488e955235fd56860f,"1950 w edgewood ave, indianapolis, in, 46217, us",61.0
2,0000902237,2237,GDYR AUTO SERVICE CTR #2237,Y,2015-10-01,NaT,5811 atlanta highway,montgomery,al,36117-2109,us,32.382199,-86.198940,36117,"5811 atlanta highway, montgomery, al, 36117, us",sg:3856d22b52984dce9b2b5d088af0c834,"3001 hayneville rd, montgomery, al, 36108, us",65.0
3,0000902382,2382,GDYR AUTO SERVICE CTR #2382,Y,2015-08-20,NaT,910 concord parkway north,concord,nc,28027-5943,us,35.424506,-80.615242,28027,"910 concord parkway north, concord, nc, 28027, us",sg:3a4f7419369244279e890530385dfa40,"8590 concord mills blvd, concord, nc, 28027, us",75.0
4,0000906740,6740,GDYR AUTO SERVICE CTR #6740,Y,2015-07-23,NaT,7320 north keystone avenue,indianapolis,in,46240-3246,us,39.888791,-86.122685,46240,"7320 north keystone avenue, indianapolis, in, ...",sg:4a8a843586cd4b51afdccd7d7481ab77,"1303 s high school rd, indianapolis, in, 46241...",69.0


In [14]:
dict_POI_lat=df_POI_GY.set_index("safegraph_place_id").to_dict()['latitude']
dict_POI_lng=df_POI_GY.set_index("safegraph_place_id").to_dict()['longitude']

df_storelist_BQ['POI_latitude']=df_storelist_BQ['poi_id'].apply(lambda x: dict_POI_lat[x])
df_storelist_BQ['POI_longitude']=df_storelist_BQ['poi_id'].apply(lambda x: dict_POI_lng[x])
df_storelist_BQ.head(2)

,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ,poi_id,poi_add,fuzz_ratio,POI_latitude,POI_longitude
0,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 s sepulveda blvd,los angeles,ca,90045-4805,us,33.953199,-118.395727,90045,"9210 s sepulveda blvd, los angeles, ca, 90045, us",sg:10999a1417bf432e818360b377cbb134,"15751 los gatos blvd, los gatos, ca, 95032, us",63.0,37.239784,-121.961645
1,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 glenwood avenue,raleigh,nc,27612-4532,us,35.837050,-78.675850,27612,"4325 glenwood avenue, raleigh, nc, 27612, us",sg:0bb487af91b544488e955235fd56860f,"1950 w edgewood ave, indianapolis, in, 46217, us",61.0,39.678604,-86.19511


In [21]:
def cal_dist_in_miles(lat_1,lng_1,lat_2,lng_2):
    dist=haversine.haversine([lat_1,lng_1],[lat_2,lng_2],unit="mi")
    return dist
df_storelist_BQ['latitude_BQ']=df_storelist_BQ['latitude_BQ'].astype(float)
df_storelist_BQ['longitude_BQ']=df_storelist_BQ['longitude_BQ'].astype(float)
df_storelist_BQ['POI_latitude']=df_storelist_BQ['POI_latitude'].astype(float)
df_storelist_BQ['POI_longitude']=df_storelist_BQ['POI_longitude'].astype(float)

df_storelist_BQ['dist_diff']=df_storelist_BQ.apply(lambda x: cal_dist_in_miles(x['latitude_BQ'],x['longitude_BQ'],x['POI_latitude'],x['POI_longitude']),axis=1)


In [22]:
df_storelist_BQ

,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ,poi_id,poi_add,fuzz_ratio,POI_latitude,POI_longitude,dist_diff
0,0000908641,8641,JUST TIRES #8641,Y,2015-06-25,NaT,9210 s sepulveda blvd,los angeles,ca,90045-4805,us,33.953199,-118.395727,90045,"9210 s sepulveda blvd, los angeles, ca, 90045, us",sg:10999a1417bf432e818360b377cbb134,"15751 los gatos blvd, los gatos, ca, 95032, us",63.0,37.239784,-121.961645,302.771289
1,0000902356,2356,JUST TIRES #2356,Y,2015-08-20,NaT,4325 glenwood avenue,raleigh,nc,27612-4532,us,35.837050,-78.675850,27612,"4325 glenwood avenue, raleigh, nc, 27612, us",sg:0bb487af91b544488e955235fd56860f,"1950 w edgewood ave, indianapolis, in, 46217, us",61.0,39.678604,-86.195110,488.768114
2,0000902237,2237,GDYR AUTO SERVICE CTR #2237,Y,2015-10-01,NaT,5811 atlanta highway,montgomery,al,36117-2109,us,32.382199,-86.198940,36117,"5811 atlanta highway, montgomery, al, 36117, us",sg:3856d22b52984dce9b2b5d088af0c834,"3001 hayneville rd, montgomery, al, 36108, us",65.0,32.348236,-86.345430,8.865360
3,0000902382,2382,GDYR AUTO SERVICE CTR #2382,Y,2015-08-20,NaT,910 concord parkway north,concord,nc,28027-5943,us,35.424506,-80.615242,28027,"910 concord parkway north, concord, nc, 28027, us",sg:3a4f7419369244279e890530385dfa40,"8590 concord mills blvd, concord, nc, 28027, us",75.0,35.374168,-80.724518,7.069241
4,0000906740,6740,GDYR AUTO SERVICE CTR #6740,Y,2015-07-23,NaT,7320 north keystone avenue,indianapolis,in,46240-3246,us,39.888791,-86.122685,46240,"7320 north keystone avenue, indianapolis, in, ...",sg:4a8a843586cd4b51afdccd7d7481ab77,"1303 s high school rd, indianapolis, in, 46241...",69.0,39.747850,-86.268722,12.445638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,0000900639,0639,GOODYEAR AUTO SERVICE - NEW LONDON,Y,2015-07-31,NaT,527 vauxhall street,new london,ct,06320-2624,us,41.369841,-72.122745,06320,"527 vauxhall street, new london, ct, 06320, us",sg:3c28eff56472464e82b9ceb5ec5200d2,"527 vauxhall st, new london, ct, 06320, us",95.0,41.370014,-72.122287,0.026587
567,0000900358,0358,GOODYEAR AUTO SERVICE - WETHERSFIELD,Y,2015-07-31,NaT,1020 silas deane highway,wethersfield,ct,06109-4231,us,41.694617,-72.656831,06109,"1020 silas deane highway, wethersfield, ct, 06...",sg:4d768c6ce85c4ef6999bbd3ff4abf3ea,"1020 silas deane hwy, wethersfield, ct, 06109, us",96.0,41.694769,-72.656546,0.018069
568,0000900629,0629,GOODYEAR AUTO SERVICE - VERNON,Y,2015-07-31,NaT,378 kelly road suite 1,vernon rockville,ct,06066-3997,us,41.825132,-72.498800,06066,"378 kelly road suite 1, vernon rockville, ct, ...",sg:3975088879a44f30948bdb7248ca58ab,"378 kelly rd ste 1, vernon rockville, ct, 0606...",96.0,41.824875,-72.498635,0.019685
569,0000900360,0360,GOODYEAR AUTO SERVICE - HARTFORD,Y,2015-07-31,NaT,306 market street,hartford,ct,06120-2918,us,41.771720,-72.670842,06120,"306 market street, hartford, ct, 06120, us",sg:c1884577280a4c69b1293bc65baf5781,"306 market st, hartford, ct, 06120, us",95.0,41.771846,-72.670220,0.033213


In [23]:
df_storelist_BQ.to_csv("./matched_QC_address.csv",index=False)

In [26]:
df_storelist_BQ[(df_storelist_BQ['fuzz_ratio']>80) & (df_storelist_BQ['dist_diff']>3)]

,installer_store_num,store_num,installer_name,company_owned_store_ind,open_date,close_date,address_1_BQ,city_BQ,state_BQ,zip_cd_BQ,country_BQ,latitude_BQ,longitude_BQ,zip_5_BQ,str_add_BQ,poi_id,poi_add,fuzz_ratio,POI_latitude,POI_longitude,dist_diff
192,0000908022,8022,GOODYEAR AUTO SERVICE - KANEOHE BAY SHOPPING C...,Y,2015-07-09,NaT,46-047 kamehameha highway,kaneohe,hi,96744-3736,us,21.379420,-157.785080,96744,"46-047 kamehameha highway, kaneohe, hi, 96744, us",sg:386f5cf15ef647799054247651827ab0,"46 047 kamehameha hwy, heeia, hi, 96744, us",85.0,21.422306,-157.803035,3.180300
355,0000902369,2369,GOODYEAR AUTO SERVICE - ST ANDREWS,Y,2015-08-20,NaT,4520 st andrews road,columbia,sc,29210-4100,us,34.051571,-81.156804,29210,"4520 st andrews road, columbia, sc, 29210, us",sg:01c053ae3e6b4d9bb6439f22edb79ec2,"6204 garners road, columbia, sc, 29209, us",83.0,33.981959,-80.963815,12.053614
480,0000902844,2844,GOODYEAR AUTO SERVICE - FREDERICKSBURG CORNER,Y,2015-06-15,NaT,9631 jefferson davis hwy,fredericksburg,va,22407-9407,us,38.268799,-77.547600,22407,"9631 jefferson davis hwy, fredericksburg, va, ...",sg:100a5fde5ee249848345999bf8c26bbc,"9631 jefferson davis hwy, fredericksburg, va, ...",100.0,38.220726,-77.506817,3.991260
520,0000902251,2251,GOODYEAR AUTO SERVICE - DESTIN,Y,2015-07-07,NaT,34721 emerald coast parkway,destin,fl,32541,us,30.393594,-86.495783,32541,"34721 emerald coast parkway, destin, fl, 32541...",sg:0ca7378603a24fb4a6e644280900048b,"34721 emerald coast pkwy, destin, fl, 32541, us",97.0,30.387416,-86.440517,3.321387


In [28]:
df_storelist_BQ[(df_storelist_BQ['fuzz_ratio']<80) & (df_storelist_BQ['dist_diff']<3)].shape

(25, 21)

In [29]:
df_storelist_BQ[(df_storelist_BQ['fuzz_ratio']<80) & (df_storelist_BQ['dist_diff']>=3)].shape

(115, 21)

In [30]:
df_storelist_BQ[(df_storelist_BQ['fuzz_ratio']<80) & (df_storelist_BQ['dist_diff']>=3)].shape

(115, 21)